In [20]:
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# 1. RE-CONFIRMACIÓN DE RUTAS
# Asegúrate de que esta ruta es la que te funcionó antes
DATA_DIR = r"C:\Users\kacam\TFM\ML - Ranking\Tablas"
WGS84 = "EPSG:4326"
UTM30N = "EPSG:25830"

# Diccionario de archivos
paths = {
    "B1": os.path.join(DATA_DIR, "B1_Restaurantes_2025_Lat_Long_CLEAN.csv"),
    "B2": os.path.join(DATA_DIR, "B2_Menu_RestaurantesMX_LIMPIO.csv"),
    "B3": os.path.join(DATA_DIR, "B3_Restaurantes_Terrazas_2025_Lat_Long_LIMPIO.csv"),
    "B4": os.path.join(DATA_DIR, "B4_Flujo_Peatones_2024_LIMPIO.csv"),
    "B5": os.path.join(DATA_DIR, "B5_Licencias_2025_Lat_Long_LIMPIO.csv"),
    "B6": os.path.join(DATA_DIR, "B6_Residentes_Edad_Nacionalidad_2025_LIMPIO.csv"),
    "B7": os.path.join(DATA_DIR, "B7_Poblacion_Madrid_LIMPIO_v2.csv"),
    "B8": os.path.join(DATA_DIR, "B8_Aparcamientos_Publicos.xlsx"),
    "B9": os.path.join(DATA_DIR, "B9_Estaciones_Metro_Renfe_CLEAN.csv"),
}

# 2. FUNCIÓN DE CARGA Y CONVERSIÓN AUTOMÁTICA
def load_to_gdf(name, path):
    # Carga el archivo
    if path.endswith('.csv'):
        df = pd.read_csv(path, encoding='utf-8')
    else:
        df = pd.read_excel(path)
    
    # Identifica columnas de coordenadas
    cols = df.columns.tolist()
    lat_col = next((c for c in cols if c.lower() in ['latitud', 'lat_num', 'lat', 'latitude']), None)
    lon_col = next((c for c in cols if c.lower() in ['longitud', 'lon_num', 'lon', 'longitude']), None)
    
    if lat_col and lon_col:
        df = df.dropna(subset=[lat_col, lon_col])
        # Limpieza de caracteres si vienen como string con comas
        df[lat_col] = pd.to_numeric(df[lat_col].astype(str).str.replace(',', '.'), errors='coerce')
        df[lon_col] = pd.to_numeric(df[lon_col].astype(str).str.replace(',', '.'), errors='coerce')
        df = df.dropna(subset=[lat_col, lon_col])
        
        geometry = [Point(xy) for xy in zip(df[lon_col], df[lat_col])]
        gdf = gpd.GeoDataFrame(df, geometry=geometry, crs=WGS84)
        return gdf.to_crs(UTM30N)
    else:
        # Si no tiene coordenadas (como B2 o B6 que son tablas de datos), devolvemos el DataFrame
        return df

# 3. EJECUCIÓN
gdfs = {}
for name, path in paths.items():
    if os.path.exists(path):
        gdfs[name] = load_to_gdf(name, path)
        print(f"✅ {name} cargado y procesado.")
    else:
        print(f"❌ {name} no encontrado en la ruta.")

print("\n--- PROCESO COMPLETADO ---")

✅ B1 cargado y procesado.
✅ B2 cargado y procesado.
✅ B3 cargado y procesado.
✅ B4 cargado y procesado.
✅ B5 cargado y procesado.
✅ B6 cargado y procesado.
✅ B7 cargado y procesado.
✅ B8 cargado y procesado.
✅ B9 cargado y procesado.

--- PROCESO COMPLETADO ---


In [22]:
from shapely.geometry import box
import numpy as np

# Creamos el área de estudio basada en los restaurantes (B1)
xmin, ymin, xmax, ymax = gdfs["B1"].total_bounds
cell_size = 150 # metros

# Generar la malla
cols = list(np.arange(xmin, xmax + cell_size, cell_size))
rows = list(np.arange(ymin, ymax + cell_size, cell_size))
polygons = [box(x, y, x + cell_size, y + cell_size) for x in cols for y in rows]

grid = gpd.GeoDataFrame({'geometry': polygons}, crs=UTM30N)
grid['grid_id'] = [f"cell_{i:06d}" for i in range(len(grid))]

# --- AQUÍ GUARDAMOS LAS COORDENADAS PARA TU MAPA ---
grid_wgs84 = grid.to_crs(WGS84)
grid['lat_center'] = grid_wgs84.geometry.centroid.y
grid['lon_center'] = grid_wgs84.geometry.centroid.x

print(f"🧱 Grid generado: {len(grid)} celdas con Lat/Lon listas para dibujo.")

🧱 Grid generado: 36292 celdas con Lat/Lon listas para dibujo.


C:\Users\kacam\AppData\Local\Temp\ipykernel_16188\572615148.py:18: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  grid['lat_center'] = grid_wgs84.geometry.centroid.y
C:\Users\kacam\AppData\Local\Temp\ipykernel_16188\572615148.py:19: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  grid['lon_center'] = grid_wgs84.geometry.centroid.x


In [24]:
def count_points_within(gdf_points, centers, radius_m):
    """Cuenta puntos dentro de un radio alrededor de centroides de forma eficiente"""
    if gdf_points is None or len(gdf_points) == 0:
        return pd.Series(0, index=centers.index)
    
    # Creamos un área de influencia (buffer) en metros
    # Usamos centers.apply para mayor precisión celda a celda
    return centers.apply(lambda c: gdf_points.within(c.buffer(radius_m)).sum())

def min_dist(points_gdf, centers):
    """Calcula la distancia al punto más cercano"""
    if points_gdf is None or len(points_gdf) == 0:
        return pd.Series(np.nan, index=centers.index)
    return centers.apply(lambda c: points_gdf.distance(c).min())

def normalize_minmax(s):
    """Normaliza una serie entre 0 y 1"""
    if s.max() == s.min():
        return s * 0
    return (s - s.min()) / (s.max() - s.min())

print("✅ Funciones espaciales cargadas correctamente.")

✅ Funciones espaciales cargadas correctamente.


In [29]:
import pandas as pd
import numpy as np

# 1. Recargamos dfs si se ha perdido (asegúrate de que DATA_DIR esté definido)
dfs = {}
for k, p in paths.items():
    if p.endswith('.csv'):
        dfs[k] = pd.read_csv(p, encoding='utf-8')
    else:
        dfs[k] = pd.read_excel(p)

# 2. Re-definimos las funciones por si acaso
def count_points_within(gdf_points, centers, radius_m):
    if gdf_points is None or len(gdf_points) == 0: return pd.Series(0, index=centers.index)
    return centers.apply(lambda c: gdf_points.within(c.buffer(radius_m)).sum())

def normalize_minmax(s):
    if s.max() == s.min(): return s * 0
    return (s - s.min()) / (s.max() - s.min())

print("✅ Datos y funciones listos.")

✅ Datos y funciones listos.


In [38]:

cent = grid.geometry.centroid

# 1. Variables Base
grid["rest_total_500m"] = count_points_within(gdfs["B1"], cent, 500)
grid["terrazas_500m"] = count_points_within(gdfs["B3"], cent, 500)
grid["licencias_500m"] = count_points_within(gdfs["B5"], cent, 500)

# 2. Identificación de Mexicanos (B1 + B2)
# Buscamos la columna de ID en B2 de forma flexible
col_id_b2 = next((c for c in dfs["B2"].columns if c.lower() in ['id_local', 'id', 'id_restaurante']), None)
col_id_b1 = next((c for c in gdfs["B1"].columns if c.lower() in ['id_local', 'id', 'id_restaurante']), None)

if col_id_b2 and col_id_b1:
    mx_ids = dfs["B2"][col_id_b2].unique()
    gdf_mx = gdfs["B1"][gdfs["B1"][col_id_b1].isin(mx_ids)]
    grid["mx_rest_total_500m"] = count_points_within(gdf_mx, cent, 500)
else:
    print("⚠️ No se pudo cruzar B1 con B2 por falta de ID. Usando 0 para mx_rest.")
    grid["mx_rest_total_500m"] = 0

# 3. Penalización por Saturación (Lógica de Negocio)
def saturation_penalty(mx_count):
    if mx_count <= 2: return 1.0  # Zona ideal (clúster saludable)
    if mx_count <= 4: return 0.7  # Riesgo de saturación
    return 0.4                   # Saturación alta: el score cae

grid["penalty_sat"] = grid["mx_rest_total_500m"].apply(saturation_penalty)


In [41]:
import geopandas as gpd
import pandas as pd
import numpy as np

print("🚀 INICIANDO GENERACIÓN DEL DATASET BLINDADO V7.1...")

# --- 1. FUNCIÓN DE LIMPIEZA AGRESIVA ---
def safe_clean(gdf):
    """Elimina columnas de sistema que rompen los sjoin"""
    if gdf is None: return None
    cols_to_drop = ['index_right', 'index_left']
    return gdf.drop(columns=[c for c in cols_to_drop if c in gdf.columns], errors='ignore').to_crs("EPSG:25830")

# Limpieza inicial de todas las capas
grid_m = safe_clean(grid)
b1_m = safe_clean(gdfs["B1"])
b3_m = safe_clean(gdfs["B3"])
b4_m = safe_clean(gdfs["B4"])
b5_m = safe_clean(gdfs["B5"])

# --- 2. ASIGNACIÓN DE DISTRITO (Con Limpieza Inmediata) ---
print("📍 Cruzando Grid con Distritos...")
b1_ref = b1_m[['desc_distrito_local', 'geometry']].copy().dropna()

# SJOIN
grid_m = gpd.sjoin(grid_m, b1_ref, how="left", predicate="intersects")

# ¡LIMPIEZA CRÍTICA AQUÍ! Borramos index_right inmediatamente
grid_m = safe_clean(grid_m) 

# Procesamos nombres
grid_m['distrito_join'] = grid_m['desc_distrito_local'].astype(str).str.upper().str.strip()
grid_m = grid_m.drop_duplicates(subset=['grid_id'])

print(f"   -> Distritos asignados. Celdas listas: {len(grid_m)}")

# --- 3. ASIGNACIÓN DE POBLACIÓN (B7) ---
print("🏠 Asignando Población...")
df_b7 = dfs["B7"].copy()
df_b7['distrito_join'] = df_b7['distrito'].astype(str).str.upper().str.strip()

# Mapa de población por distrito
pop_map = df_b7.groupby('distrito_join')['num_personas'].sum().to_dict()
cells_per_dist = grid_m['distrito_join'].value_counts().to_dict()

# Función de reparto segura
def get_pop(d):
    return pop_map.get(d, 0) / cells_per_dist.get(d, 1) if d in pop_map else 0

grid_m['poblacion_estimada'] = grid_m['distrito_join'].apply(get_pop)

# --- 4. VARIABLES DE CONTEXTO (Función Blindada) ---
print("⚙️ Calculando densidades (Terrazas, Licencias)...")

def count_in_radius_safe(points_gdf, grid_gdf, radius=500):
    # Limpiamos AMBOS dataframes antes de tocar nada
    pts = safe_clean(points_gdf)
    centers = safe_clean(grid_gdf).copy() # Copia limpia
    
    # Creamos buffer
    centers['geometry'] = centers.geometry.centroid.buffer(radius)
    
    # Join seguro
    joined = gpd.sjoin(pts, centers, how="inner", predicate="within")
    
    return joined.groupby("grid_id").size()

# Ejecutamos los conteos
grid_m['terrazas_500m'] = grid_m['grid_id'].map(count_in_radius_safe(b3_m, grid_m)).fillna(0)
grid_m['licencias_500m'] = grid_m['grid_id'].map(count_in_radius_safe(b5_m, grid_m)).fillna(0)
grid_m['rest_total_500m'] = grid_m['grid_id'].map(count_in_radius_safe(b1_m, grid_m)).fillna(0)

# --- 5. PEATONES (B4) ---
print("🚶 Calculando Peatones...")
# Agrupar por ubicación única para no duplicar sensores
b4_unique = b4_m.groupby('geometry')['cantidad_peatones'].mean().reset_index()
b4_unique = gpd.GeoDataFrame(b4_unique, geometry='geometry', crs="EPSG:25830")

# Usamos la función de conteo para saber cuántos sensores hay cerca
# (Si quieres sumar el valor del flujo, habría que hacer un sjoin específico, 
#  pero para el modelo, la "intensidad peatonal" se puede inferir del conteo y del score final)
centers_temp = safe_clean(grid_m).copy()
centers_temp['geometry'] = centers_temp.geometry.centroid.buffer(500)
joined_b4 = gpd.sjoin(b4_unique, centers_temp, how="inner", predicate="within")

# Sumamos el valor de flujo, no solo contamos puntos
grid_m['peatones_val'] = grid_m['grid_id'].map(joined_b4.groupby("grid_id")['cantidad_peatones'].sum()).fillna(0)

# --- 6. SCORE FINAL (TARGET) ---
print("🎯 Calculando Score Objetivo...")
def norm(s):
    return (s - s.min()) / (s.max() - s.min() + 1e-9)

grid_m['score_final'] = (
    0.5 * norm(grid_m['peatones_val']) + 
    0.5 * norm(grid_m['poblacion_estimada'])
).clip(0, 1)

# --- 7. EXPORTACIÓN ---
# Variables PROHIBIDAS para el entrenamiento (Leakage) -> peatones_val, poblacion_estimada
# Variables PERMITIDAS -> terrazas_500m, licencias_500m, rest_total_500m, dist_metro (si la tienes)

output_file = "Dataset_Madrid_Model_Blind_V7.csv"
grid_m.to_csv(output_file, index=False)

print(f"\n✅ ¡ÉXITO! Dataset generado: {output_file}")
print(f"   - Filas totales: {len(grid_m)}")
print(f"   - Filas con Score > 0: {len(grid_m[grid_m['score_final'] > 0])}")
print(f"   - Media de Terrazas: {grid_m['terrazas_500m'].mean():.2f}")

🚀 INICIANDO GENERACIÓN DEL DATASET BLINDADO V7.1...
📍 Cruzando Grid con Distritos...
   -> Distritos asignados. Celdas listas: 36292
🏠 Asignando Población...
⚙️ Calculando densidades (Terrazas, Licencias)...
🚶 Calculando Peatones...
🎯 Calculando Score Objetivo...

✅ ¡ÉXITO! Dataset generado: Dataset_Madrid_Model_Blind_V8.csv
   - Filas totales: 36292
   - Filas con Score > 0: 3984
   - Media de Terrazas: 6.04
